In [1]:
import sys
sys.path.append('/work/utilis')

from lib_to_use import *
from general_functions import *
from paths import *

Torch version:  2.6.0+cu124
/root/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Pyterrier version:  0.13.0
/work/utilis/lib_to_use.py:26: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [2]:
# # Load datasets
df_train = load_jsonl(train_path)
df_val = load_jsonl(val_path)
df_test = load_jsonl(test_path)

In [3]:
start_time = time.time()
df_train["text"] = df_train["text"].apply(text_processing_pipeline)
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_train_processed.pkl", "wb") as f:
     pickle.dump(df_train, f)
print(f"Elapsed time: {time.time() - start_time:.2f} seconds")

In [5]:
start_time = time.time()
df_val["text"] = df_val["text"].apply(text_processing_pipeline)
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_val_processed.pkl", "wb") as f:
     pickle.dump(df_val, f)
print(f"Elapsed time: {time.time() - start_time:.2f} seconds")

In [6]:
start_time = time.time()
df_test["text"] = df_test["text"].apply(text_processing_pipeline)
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_test_processed.pkl", "wb") as f:
     pickle.dump(df_test, f)
print(f"Elapsed time: {time.time() - start_time:.2f} seconds")

In [7]:
# # open the pickle file df_answers_train
# with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_answers_test.pkl", "rb") as f:
#     answers_test = pickle.load(f)
# answers_test

In [8]:
start_time = time.time()
df_answers_val = load_subset(answers_path, subset_size=500000, chunk_size=100000, split="val")
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_answers_val.pkl", "wb") as f:
     pickle.dump(df_answers_val, f)
print(f"Elapsed time: {time.time() - start_time:.2f} seconds")

In [9]:
start_time = time.time()
df_answers_test = load_subset(answers_path, subset_size=500000, chunk_size=100000, split="test")
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_answers_test.pkl", "wb") as f:
     pickle.dump(df_answers_test, f)
print(f"Elapsed time: {time.time() - start_time:.2f} seconds")

In [10]:
start_time = time.time()
df_answers_train1 = load_subset(answers_path, subset_size=1000000, chunk_size=100000, split="train1")
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_answers_train1.pkl", "wb") as f:
     pickle.dump(df_answers_train1, f)
print(f"Elapsed time: {time.time() - start_time:.2f} seconds")
df_answers_train1

In [11]:
start_time = time.time()
df_answers_train2 = load_subset(answers_path, subset_size=2000000, chunk_size=100000, split="train2")
print(f"Elapsed time: {time.time() - start_time:.2f} seconds")

In [12]:
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_answers_train2.pkl", "wb") as f:
     pickle.dump(df_answers_train2, f)

In [13]:
with open(test_processed_path, "rb") as f:
    df_test = pickle.load(f)
df_test = merge_answers_in_chunks(df_test, test_answers_path)

In [14]:
df_test["best_answer_text"] = df_test["best_answer_text"].apply(text_processing_pipeline)
df_test
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_test_processed.pkl", "wb") as f:
     pickle.dump(df_test, f)

In [15]:
with open(val_processed_path, "rb") as f:
    df_val = pickle.load(f)
df_val = merge_answers_in_chunks(df_val, val_answers_path)
df_val["best_answer_text"] = df_val["best_answer_text"].apply(text_processing_pipeline)
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_val_processed.pkl", "wb") as f:
     pickle.dump(df_val, f)

In [16]:
with open(train_processed_path, "rb") as f:
    df_train = pickle.load(f)
df_train = merge_answers_in_chunks(df_train, train_answers1_path)

In [17]:
df_train = merge_answers_in_chunks(df_train, train_answers2_path)

We applied the <code>merge_answers_in_chunks</code> function twice, resulting in pandas adding **"_x"** and **"_y"** to the column names to properly distinguish them.


<img src="image-20250207-155522.png" width="75%" align="" />

Let's now merge the columns <code>best_answer_text</code> and `id`by discarding the NaN values and apply the <code>text_processing_pipeline</code> function to <code>best_answer_text</code>.

In [18]:
# Unisci le due colonne in base ai valori non NaN
df_train["best_answer_text"] = df_train["best_answer_text_x"].combine_first(df_train["best_answer_text_y"])
df_train["Id"] = df_train["Id_x"].combine_first(df_train["Id_y"])

# Ora rimuovi le colonne ridondanti
df_train.drop(columns=["best_answer_text_x", "best_answer_text_y"], inplace=True)
df_train.drop(columns=["Id_x", "Id_y"], inplace=True)

In [19]:
df_train["best_answer_text"] = df_train["best_answer_text"].apply(text_processing_pipeline)

<img src="image-20250207-160200.png" width="" align="" />

In [20]:
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_train_processed.pkl", "wb") as f:
     pickle.dump(df_train, f)

In [21]:
def processing_to_index(df):
    # rename features for qid, query, docno, text
    df.rename(columns={"id": "qid", "text": "query", "Id": "docno", "best_answer_text": "text"}, inplace=True)

    # remove missing "text" or "docno"
    df.dropna(subset=["text", "docno"], inplace=True)

    #remove empty "text" and "docno"
    df = df[df["text"].str.strip() != ""] 
    df = df[df["docno"].str.strip() != ""]
    return df

In [22]:
with open(train_processed_path, "rb") as f:
    df_train = pickle.load(f)
df_train = processing_to_index(df_train)

print("Empty docno:", (df_train['docno'].str.strip() == "").sum())
print("Empty text:", (df_train['text'].str.strip() == "").sum())

#remove judaism_44648 and judaism_2117 due to characters utilized
df_train = df_train[df_train["docno"] != "judaism_2117"]
df_train = df_train[df_train["docno"] != "judaism_44648"]
df_train

In [23]:
with open(val_processed_path, "rb") as f:
    df_val = pickle.load(f)
df_val = processing_to_index(df_val)
print("Empty docno:", (df_val['docno'].str.strip() == "").sum())
print("Empty text:", (df_val['text'].str.strip() == "").sum())
df_val

In [24]:
with open(test_processed_path, "rb") as f:
    df_test = pickle.load(f)
df_test = processing_to_index(df_test)
print("Empty docno:", (df_test['docno'].str.strip() == "").sum())
print("Empty text:", (df_test['text'].str.strip() == "").sum())
df_test

In [25]:
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_train_processed.pkl", "wb") as f:
     pickle.dump(df_train, f)
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_val_processed.pkl", "wb") as f:
     pickle.dump(df_val, f)
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_test_processed.pkl", "wb") as f:
     pickle.dump(df_test, f)

In [26]:
# Combina tutti i dataset in un unico DataFrame
df_all = pd.concat([df_train, df_val, df_test], ignore_index=True)
df_all
# Se necessario, elimina eventuali duplicati nel campo "docno"
df_all.drop_duplicates(subset="docno", inplace=True)
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/df_all_processed.pkl", "wb") as f:
     pickle.dump(df_all, f)
df_all

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=144ec01f-394f-474f-b507-b786ab13b472' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>